# Creating agents with models not yet optimized for Bedrock Agents

In this notebook you will learn how to create an Amazon Bedrock Agent using the Mistral Large model. We will adapt the restaurant agent created before. The architecture looks as following:

<img src="images/architecture.png" style="width:70%;display:block;margin: 0 auto;">
<br/>

The steps to complete this notebook are:

1. Import the needed libraries
1. Create the Knowledge Base for Amazon Bedrock
1. Upload the dataset to Amazon S3
1. Create the Amazon Bedrock Agent
1. Test the Agent
1. Clean-up the resources created

## 1. Import the needed libraries

First step is to install the pre-requisites packages

In [1]:
!pip install --upgrade -q -r requirements.txt

In [2]:
import os
import time
import boto3
import logging
import pprint
import json

from knowledge_base import BedrockKnowledgeBase
from agent import create_agent_role_and_policies, create_lambda_role, delete_agent_roles_and_policies
from agent import create_dynamodb, create_lambda, clean_up_resources

In [3]:
#Clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
region, account_id

('us-east-1', '061051260563')

In [4]:
suffix = f"{region}-{account_id}"
agent_name = 'restaurant-agent-mistral'
knowledge_base_name = f'{agent_name}-kb'
knowledge_base_description = "Knowledge Base containing the restaurant menu's collection"
agent_alias_name = "booking-ag3nt-alias"
bucket_name = f'{agent_name}-{suffix}'
agent_bedrock_allow_policy_name = f"{agent_name}-ba"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'
agent_foundation_model = "mistral.mistral-large-2407-v1:0"

agent_description = "Agent in charge of a restaurants table bookings"
agent_instruction = """
You are a restaurant agent, helping clients retrieve information from their booking, 
create a new booking or delete an existing booking
"""

agent_action_group_description = """
Actions for getting table booking information, create a new booking or delete an existing booking"""

agent_action_group_name = "TableBookingsActionGroup"

Specify foundational model of choice below. 

To specify this, update the foundational model id for the specific model you would like to choose.
- **Amazon Nova Pro**: us.amazon.nova-pro-v1:0
- **Anthropic Claude 3.5 Sonnet v2**: us.anthropic.claude-3-sonnet-20240229-v1:0

In [5]:
# agent_foundation_model = "us.anthropic.claude-3-sonnet-20240229-v1:0"
agent_foundation_model = "us.amazon.nova-pro-v1:0"
foundation_model = agent_foundation_model[3:] 

print(foundation_model)

amazon.nova-pro-v1:0


## 2. Create Amazon Bedrock Knowledge Base
Let's start by creating a [Amazon Bedrock Knowledge Base](https://aws.amazon.com/bedrock/knowledge-bases/) to store the restaurant menus. Knowledge Bases allow you to integrate with different vector databases including [Amazon OpenSearch Serverless](https://aws.amazon.com/opensearch-service/features/serverless/), [Amazon Aurora](https://aws.amazon.com/rds/aurora/) and [Pinecone](http://app.pinecone.io/bedrock-integration). For this example, we will integrate the knowledge base with Amazon OpenSearch Serverless. To do so, we will use the helper class `BedrockKnowledgeBase` which will create the knowledge base and all of its pre-requisites:
1. IAM roles and policies
2. S3 bucket
3. Amazon OpenSearch Serverless encryption, network and data access policies
4. Amazon OpenSearch Serverless collection
5. Amazon OpenSearch Serverless vector index
6. Knowledge base
7. Knowledge base data source

In [6]:
knowledge_base = BedrockKnowledgeBase(
    kb_name=knowledge_base_name,
    kb_description=knowledge_base_description,
    data_bucket_name=bucket_name
)

[2025-04-22 10:22:00,300] p25552 {credentials.py:1352} INFO - Found credentials in shared credentials file: ~/.aws/credentials
[2025-04-22 10:22:00,780] p25552 {credentials.py:1352} INFO - Found credentials in shared credentials file: ~/.aws/credentials


Step 1 - Creating or retrieving restaurant-agent-mistral-us-east-1-061051260563 S3 bucket for Knowledge Base documents
Creating bucket restaurant-agent-mistral-us-east-1-061051260563
Step 2 - Creating Knowledge Base Execution Role (AmazonBedrockExecutionRoleForKnowledgeBase_061051) and Policies
Step 3 - Creating OSS encryption, network and data access policies
Step 4 - Creating OSS Collection (this step takes a couple of minutes to complete)
{ 'ResponseMetadata': { 'HTTPHeaders': { 'connection': 'keep-alive',
                                         'content-length': '317',
                                         'content-type': 'application/x-amz-json-1.0',
                                         'date': 'Tue, 22 Apr 2025 15:22:02 '
                                                 'GMT',
                                         'x-amzn-requestid': '0f2b2b41-9d21-4efb-9b49-b973bcb8f99d'},
                        'HTTPStatusCode': 200,
                        'RequestId': '0f2b2b41-9d

[2025-04-22 10:23:34,179] p25552 {base.py:258} INFO - PUT https://6a0mp0nmj62w5r1kqeel.us-east-1.aoss.amazonaws.com:443/bedrock-sample-rag-index-061051 [status:200 request:0.685s]



Creating index:
{ 'acknowledged': True,
  'index': 'bedrock-sample-rag-index-061051',
  'shards_acknowledged': True}
Step 6 - Creating Knowledge Base
{ 'createdAt': datetime.datetime(2025, 4, 22, 15, 24, 34, 762874, tzinfo=tzutc()),
  'description': "Knowledge Base containing the restaurant menu's collection",
  'knowledgeBaseArn': 'arn:aws:bedrock:us-east-1:061051260563:knowledge-base/LD5AKXTEHU',
  'knowledgeBaseConfiguration': { 'type': 'VECTOR',
                                  'vectorKnowledgeBaseConfiguration': { 'embeddingModelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-embed-text-v2:0'}},
  'knowledgeBaseId': 'LD5AKXTEHU',
  'name': 'restaurant-agent-mistral-kb',
  'roleArn': 'arn:aws:iam::061051260563:role/AmazonBedrockExecutionRoleForKnowledgeBase_061051',
  'status': 'CREATING',
  'storageConfiguration': { 'opensearchServerlessConfiguration': { 'collectionArn': 'arn:aws:aoss:us-east-1:061051260563:collection/6a0mp0nmj62w5r1kqeel',
                      

## 3. Upload the dataset to Amazon S3
Now that we have created the knowledge base, let's populate it with the menu's dataset. The Knowledge Base data source expects the data to be available on the S3 bucket connected to it and changes on the data can be syncronized to the knowledge base using the `StartIngestionJob` API call. In this example we will use the [boto3 abstraction](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/start_ingestion_job.html) of the API, via our helper classe. 

Let's first upload the menu's data available on the `dataset` folder to s3

In [7]:
def upload_directory(path, bucket_name):
        for root,dirs,files in os.walk(path):
            for file in files:
                file_to_upload = os.path.join(root,file)
                print(f"uploading file {file_to_upload} to {bucket_name}")
                s3_client.upload_file(file_to_upload,bucket_name,file)

upload_directory("dataset", bucket_name)

uploading file dataset/Restaurant_Childrens_Menu.pdf to restaurant-agent-mistral-us-east-1-061051260563
uploading file dataset/Restaurant_Dinner_Menu.pdf to restaurant-agent-mistral-us-east-1-061051260563
uploading file dataset/Restaurant_week_specials.pdf to restaurant-agent-mistral-us-east-1-061051260563


Now we start the ingestion job

In [8]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base.start_ingestion_job()

{ 'dataSourceId': 'ZUJCRPEXEO',
  'ingestionJobId': 'M2IUUFD1MK',
  'knowledgeBaseId': 'LD5AKXTEHU',
  'startedAt': datetime.datetime(2025, 4, 22, 15, 25, 7, 695830, tzinfo=tzutc()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 0,
                  'numberOfMetadataDocumentsModified': 0,
                  'numberOfMetadataDocumentsScanned': 0,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 0},
  'status': 'STARTING',
  'updatedAt': datetime.datetime(2025, 4, 22, 15, 25, 7, 695830, tzinfo=tzutc())}
{ 'dataSourceId': 'ZUJCRPEXEO',
  'ingestionJobId': 'M2IUUFD1MK',
  'knowledgeBaseId': 'LD5AKXTEHU',
  'startedAt': datetime.datetime(2025, 4, 22, 15, 25, 7, 695830, tzinfo=tzutc()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 3,
           

Finally we collect the Knowledge Base Id to integrate it with our Agent later on

In [9]:
kb_id = knowledge_base.get_knowledge_base_id()

'LD5AKXTEHU'


### 3.1 Test the Knowledge Base
Now the Knowlegde Base is available we can test it out using the [**retrieve**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve.html) and [**retrieve_and_generate**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html) functions. 

#### Testing Knowledge Base with Retrieve and Generate API

Let's first test the knowledge base using the retrieve and generate API. With this API, Bedrock takes care of retrieving the necessary references from the knowledge base and generating the final answer using a LLM model from Bedrock

In [12]:
response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": "Which are the 5 mains available in the childrens menu?"
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region, foundation_model),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

The five mains available in the children's menu are:

1. Chicken Nuggets 2. Macaroni and Cheese 3. Grilled Cheese Sandwich 4. Mini Burgers 5. Veggie Pasta Please note that the fifth main, Veggie Pasta, is inferred from the context as the search results only explicitly list four items but mention that there are five mains available.



As you can see, with the retrieve and generate API we get the final response directly and we don't see the different sources used to generate this response. Let's now retrieve the source information from the knowledge base with the retrieve API.

#### Testing Knowledge Base with Retrieve API
If you need an extra layer of control, you can retrieve the chuncks that best match your query using the retrieve API. In this setup, we can configure the desired number of results and control the final answer with your own application logic. The API then provides you with the matching content, its S3 location, the similarity score and the chunk metadata

In [13]:
response_ret = bedrock_agent_runtime_client.retrieve(
    knowledgeBaseId=kb_id, 
    nextToken='string',
    retrievalConfiguration={
        "vectorSearchConfiguration": {
            "numberOfResults":5,
        } 
    },
    retrievalQuery={
        'text': 'Which are the 5 mains available in the childrens menu?'
    }
)

def response_print(retrieve_resp):
#structure 'retrievalResults': list of contents. Each list has content, location, score, metadata
    for num,chunk in enumerate(response_ret['retrievalResults'],1):
        print(f'Chunk {num}: ',chunk['content']['text'],end='\n'*2)
        print(f'Chunk {num} Location: ',chunk['location'],end='\n'*2)
        print(f'Chunk {num} Score: ',chunk['score'],end='\n'*2)
        print(f'Chunk {num} Metadata: ',chunk['metadata'],end='\n'*2)

response_print(response_ret)

Chunk 1:  The Regrettable Experience — Children's Menu Entrees:     1. CHICKEN NUGGETS     ●     ●     ●     Description: Crispy chicken nuggets served with a side of ketchup or ranch dressing.     Allergens: Gluten (in the coating), possible Soy.     Suitable for Vegetarians: No     2. MACARONI AND CHEESE     ●     ●     ●     Description: Classic macaroni pasta smothered in creamy cheese sauce.     Allergens: Dairy, Gluten.     Suitable for Vegetarians: Yes     3.

Chunk 1 Location:  {'s3Location': {'uri': 's3://restaurant-agent-mistral-us-east-1-061051260563/Restaurant_Childrens_Menu.pdf'}, 'type': 'S3'}

Chunk 1 Score:  0.42157477

Chunk 1 Metadata:  {'x-amz-bedrock-kb-source-uri': 's3://restaurant-agent-mistral-us-east-1-061051260563/Restaurant_Childrens_Menu.pdf', 'x-amz-bedrock-kb-document-page-number': 1.0, 'x-amz-bedrock-kb-chunk-id': '1%3A0%3AJ2MaXpYB3FIGyGrRbbwZ', 'x-amz-bedrock-kb-data-source-id': 'ZUJCRPEXEO'}

Chunk 2:  with Balsamic Vinaigrette Allergens: Potential Tree 

## 4. Create the Agent for Amazon Bedrock

In this section we will go through all the steps to create an Agent for Amazon Bedrock. 

These are the steps to complete:
    
1. Create an Amazon DynamoDB table
2. Create an AWS Lambda function
3. Create the IAM policies needed for the Agent
4. Create the Agent
5. Create the Agent Action Group
6. Allow the Agent to invoke the Action Group Lambda
7. Associate the Knowledge Base to the agent
8. Prepare the Agent and create an alias

### 4.1 Create the DynamoDB table
We will create a DynamoDB table which contains the restaurant bookings information.

In [14]:
table_name = 'restaurant_assistant'
create_dynamodb(table_name)

Creating table restaurant_assistant...
Table restaurant_assistant created successfully!


### 4.2 Create the Lambda Function

We will now create a lambda function that interacts with DynamoDB table. To do so we will:

1. Create the `lambda_function.py` file which contains the logic for our lambda function
2. Create the IAM role for our Lambda function
3. Create the lambda function with the required permissions

#### Create the function code
When creating an Agent for Amazon Bedrock, you can connect a Lambda function to the Action Group in order to execute the functions required by the agent. In this option, your agent is responsible for the execution of your functions. Let's create the lambda function tha implements the functions for `get_booking_details`, `create_booking` and `delete_booking`

In [15]:
%%writefile lambda_function.py
import json
import uuid
import boto3

dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('restaurant_assistant')

def get_named_parameter(event, name):
    """
    Get a parameter from the lambda event
    """
    return next(item for item in event['parameters'] if item['name'] == name)['value']


def get_booking_details(booking_id):
    """
    Retrieve details of a restaurant booking
    
    Args:
        booking_id (string): The ID of the booking to retrieve
    """
    try:
        response = table.get_item(Key={'booking_id': booking_id})
        if 'Item' in response:
            return response['Item']
        else:
            return {'message': f'No booking found with ID {booking_id}'}
    except Exception as e:
        return {'error': str(e)}


def create_booking(date, name, hour, num_guests):
    """
    Create a new restaurant booking
    
    Args:
        date (string): The date of the booking
        name (string): Name to idenfity your reservation
        hour (string): The hour of the booking
        num_guests (integer): The number of guests for the booking
    """
    try:
        booking_id = str(uuid.uuid4())[:8]
        table.put_item(
            Item={
                'booking_id': booking_id,
                'date': date,
                'name': name,
                'hour': hour,
                'num_guests': num_guests
            }
        )
        return {'booking_id': booking_id}
    except Exception as e:
        return {'error': str(e)}


def delete_booking(booking_id):
    """
    Delete an existing restaurant booking
    
    Args:
        booking_id (str): The ID of the booking to delete
    """
    try:
        response = table.delete_item(Key={'booking_id': booking_id})
        if response['ResponseMetadata']['HTTPStatusCode'] == 200:
            return {'message': f'Booking with ID {booking_id} deleted successfully'}
        else:
            return {'message': f'Failed to delete booking with ID {booking_id}'}
    except Exception as e:
        return {'error': str(e)}
    

def lambda_handler(event, context):
    # get the action group used during the invocation of the lambda function
    actionGroup = event.get('actionGroup', '')
    
    # name of the function that should be invoked
    function = event.get('function', '')
    
    # parameters to invoke function with
    parameters = event.get('parameters', [])

    if function == 'get_booking_details':
        booking_id = get_named_parameter(event, "booking_id")
        if booking_id:
            response = str(get_booking_details(booking_id))
            responseBody = {'TEXT': {'body': json.dumps(response)}}
        else:
            responseBody = {'TEXT': {'body': 'Missing booking_id parameter'}}

    elif function == 'create_booking':
        date = get_named_parameter(event, "date")
        name = get_named_parameter(event, "name")
        hour = get_named_parameter(event, "hour")
        num_guests = get_named_parameter(event, "num_guests")

        if date and hour and num_guests:
            response = str(create_booking(date, name, hour, num_guests))
            responseBody = {'TEXT': {'body': json.dumps(response)}}
        else:
            responseBody = {'TEXT': {'body': 'Missing required parameters'}}

    elif function == 'delete_booking':
        booking_id = get_named_parameter(event, "booking_id")
        if booking_id:
            response = str(delete_booking(booking_id))
            responseBody = {'TEXT': {'body': json.dumps(response)}}
        else:
            responseBody = {'TEXT': {'body': 'Missing booking_id parameter'}}

    else:
        responseBody = {'TEXT': {'body': 'Invalid function'}}

    action_response = {
        'actionGroup': actionGroup,
        'function': function,
        'functionResponse': {
            'responseBody': responseBody
        }
    }

    function_response = {'response': action_response, 'messageVersion': event['messageVersion']}
    print("Response: {}".format(function_response))

    return function_response

Overwriting lambda_function.py


#### Create the required permissions
Now let's also create the lambda role and its required policies. For this case, we need the lambda to be able to access DynamoDB, that is why we also create a DynamoDB policy and attach to our Lambda. To do so, we will use the support function `create_lambda_role`.

In [16]:
lambda_iam_role = create_lambda_role(agent_name, table_name)

#### Create the function

Now that we have the Lambda function code and its execution role, let's package it into a Zip file and create the Lambda resources

In [17]:
lambda_function_name = f'{agent_name}-lambda'

In [18]:
lambda_function = create_lambda(lambda_function_name, lambda_iam_role)

### 4.3 Create the IAM policies needed for the Agent

Now that we have created the Knowledge Base, our DynamoDB table and the Lambda function to execute the tasks for our agent, let's start creating our Agent.


First need to create the agent policies that allow bedrock model invocation and Knowledge Base query and the agent IAM role with the policy associated to it. We will allow this agent to invoke the Claude Sonnet model. Here we use the `create_agent_role_and_policies` to create the agent role and its required policies

In [19]:
agent_role = create_agent_role_and_policies(agent_name, agent_foundation_model, foundation_model, kb_id=kb_id)

In [20]:
agent_role

{'Role': {'Path': '/',
  'RoleName': 'AmazonBedrockExecutionRoleForAgents_restaurant-agent-mistral',
  'RoleId': 'AROAQ4NXQJ2J3ZVYLWROF',
  'Arn': 'arn:aws:iam::061051260563:role/AmazonBedrockExecutionRoleForAgents_restaurant-agent-mistral',
  'CreateDate': datetime.datetime(2025, 4, 22, 15, 27, 35, tzinfo=tzutc()),
  'AssumeRolePolicyDocument': {'Version': '2012-10-17',
   'Statement': [{'Effect': 'Allow',
     'Principal': {'Service': 'bedrock.amazonaws.com'},
     'Action': 'sts:AssumeRole'}]}},
 'ResponseMetadata': {'RequestId': '8a6b0aee-9a5f-419f-85c8-730a94e3c6ef',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 22 Apr 2025 15:27:35 GMT',
   'x-amzn-requestid': '8a6b0aee-9a5f-419f-85c8-730a94e3c6ef',
   'content-type': 'text/xml',
   'content-length': '875'},
  'RetryAttempts': 0}}

### 4.4 Create the Agent
Amazon Bedrock Agents now supports all models from Amazon Bedrock. You can now create agents with any foundation model. Currently, some of the offered models are optimized with prompts/parsers fine-tuned for integrating with the agents architecture. Over time, we plan to offer optimization for all of the offered models.

If you’ve selected a model for which optimization is not yet available, you can override the prompts to extract better responses, and if needed, override the parsers. See [Modify parser Lambda function in Amazon Bedrock Agents](https://docs.aws.amazon.com/bedrock/latest/userguide/lambda-parser.html) for more information.

As Mistral Large has not yet been optimized, we will override the orchestration prompt for better responses. The prompts used by the not yet optimized models use the [Bedrock Converse API](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_Converse.html) format with a system prompt as well the messages format. You can use [placeholder variables](https://docs.aws.amazon.com/bedrock/latest/userguide/prompt-placeholders.html) supported by Bedrock Agents in your prompt. We will use `$instruction$`, `$knowledge_base_additional_guideline$`, `$prompt_session_attributes$`, `$question$` and `$agent_scratchpad$` to overwrite our orchestration prompt

In [22]:
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=agent_foundation_model,
    instruction=agent_instruction,
    promptOverrideConfiguration={
        'promptConfigurations': [
            {
                'basePromptTemplate': """{
        "system": "
$instruction$
You are a helpful assistant with tool calling capabilities.

Given the following functions, please respond with a JSON for a function call with its proper arguments that best answers the given prompt.

Provide your final answer to the user's question within <answer></answer> xml tags.
$knowledge_base_additional_guideline$
$prompt_session_attributes$
",
        "messages": [
            {
                "role" : "user",
                "content": [{
                    "text": "$question$"
                }]
            },
            {
                "role" : "assistant",
                "content" : [{
                    "text": "$agent_scratchpad$"
                }]
            }
        ]
    }""",
                'inferenceConfiguration': {
                    'maximumLength': 2048,
                    'temperature': 0,
                    'topK': 250,
                    'topP': 1
                },
                'parserMode': 'DEFAULT',
                'promptCreationMode': 'OVERRIDDEN',
                'promptState': 'ENABLED',
                'promptType': 'ORCHESTRATION'
            }
        ]
    }
)
response

{'ResponseMetadata': {'RequestId': 'a1d66201-ec3b-4793-8da8-52c21ddedf1b',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Tue, 22 Apr 2025 15:27:53 GMT',
   'content-type': 'application/json',
   'content-length': '1886',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'a1d66201-ec3b-4793-8da8-52c21ddedf1b',
   'x-amz-apigw-id': 'JblK8El8IAMEcTw=',
   'x-amzn-trace-id': 'Root=1-6807b578-594735fe72ef466b4dbac9d0'},
  'RetryAttempts': 0},
 'agent': {'agentArn': 'arn:aws:bedrock:us-east-1:061051260563:agent/PXINSLJAOL',
  'agentCollaboration': 'DISABLED',
  'agentId': 'PXINSLJAOL',
  'agentName': 'restaurant-agent-mistral',
  'agentResourceRoleArn': 'arn:aws:iam::061051260563:role/AmazonBedrockExecutionRoleForAgents_restaurant-agent-mistral',
  'agentStatus': 'CREATING',
  'createdAt': datetime.datetime(2025, 4, 22, 15, 27, 53, 73716, tzinfo=tzutc()),
  'description': 'Agent in charge of a restaurants table bookings',
  'foundationModel': 'us.amazon.nova-pro-v1:0',
  'idleSessi

Let's get our Agent ID. It will be important to perform operations with our agent

In [23]:
agent_id = response['agent']['agentId']
print("The agent id is:",agent_id)

The agent id is: PXINSLJAOL


### 4.5 Create the Agent Action Group
We will now create an agent action group that uses the lambda function created before. The [`create_agent_action_group`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/create_agent_action_group.html) function provides this functionality. We will use `DRAFT` as the agent version since we haven't yet created an agent version or alias. To inform the agent about the action group functionalities, we will provide an action group description containing the functionalities of the action group.

In this example, we will provide the Action Group functionality using a [`functionSchema`](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-action-function.html).

To define the functions using a function schema, you need to provide the `name`, `description` and `parameters` for each function.

In [24]:
agent_functions = [
    {
        'name': 'get_booking_details',
        'description': 'Retrieve details of a restaurant booking',
        'parameters': {
            "booking_id": {
                "description": "The ID of the booking to retrieve",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        'name': 'create_booking',
        'description': 'Create a new restaurant booking',
        'parameters': {
            "date": {
                "description": "The date of the booking",
                "required": True,
                "type": "string"
            },
            "name": {
                "description": "Name to idenfity your reservation",
                "required": True,
                "type": "string"
            },
            "hour": {
                "description": "The hour of the booking",
                "required": True,
                "type": "string"
            },
            "num_guests": {
                "description": "The number of guests for the booking",
                "required": True,
                "type": "integer"
            }
        }
    },
    {
        'name': 'delete_booking',
        'description': 'Delete an existing restaurant booking',
        'parameters': {
            "booking_id": {
                "description": "The ID of the booking to delete",
                "required": True,
                "type": "string"
            }
        }
    },
]

We now use the function schema to create the agent action group using the [`create_agent_action_group`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/create_agent_action_group.html) API

In [25]:
# Pause to make sure agent is created
time.sleep(30)

# Now, we can configure and create an action group here:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    actionGroupName=agent_action_group_name,
    functionSchema={
        'functions': agent_functions
    },
    description=agent_action_group_description
)

In [26]:
agent_action_group_response

{'ResponseMetadata': {'RequestId': 'b4b322fa-3a68-4e0b-87bc-133498cc34e3',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 22 Apr 2025 15:28:30 GMT',
   'content-type': 'application/json',
   'content-length': '1456',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'b4b322fa-3a68-4e0b-87bc-133498cc34e3',
   'x-amz-apigw-id': 'JblQwGLKIAMEa_w=',
   'x-amzn-trace-id': 'Root=1-6807b59e-0a9af5473831036365666b64'},
  'RetryAttempts': 0},
 'agentActionGroup': {'actionGroupExecutor': {'lambda': 'arn:aws:lambda:us-east-1:061051260563:function:restaurant-agent-mistral-lambda'},
  'actionGroupId': 'CM7K8YKRCR',
  'actionGroupName': 'TableBookingsActionGroup',
  'actionGroupState': 'ENABLED',
  'agentId': 'PXINSLJAOL',
  'agentVersion': 'DRAFT',
  'createdAt': datetime.datetime(2025, 4, 22, 15, 28, 30, 89077, tzinfo=tzutc()),
  'description': '\nActions for getting table booking information, create a new booking or delete an existing booking',
  'functionSchema': {'functions': [{'d

### 4.6 Allow the Agent to invoke the Action Group Lambda
Before using the action group, we need to allow the agent to invoke the lambda function associated with the action group. This is done via [resource-based policy](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-permissions.html#agents-permissions-lambda). Let's add the resource-based policy to the lambda function created

In [27]:
# Create allow to invoke permission on lambda
lambda_client = boto3.client('lambda')
response = lambda_client.add_permission(
    FunctionName=lambda_function_name,
    StatementId='allow_bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
)


In [28]:
response

{'ResponseMetadata': {'RequestId': '2eb99fe0-520f-499c-9ff9-33c15323d9fb',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Tue, 22 Apr 2025 15:28:30 GMT',
   'content-type': 'application/json',
   'content-length': '359',
   'connection': 'keep-alive',
   'x-amzn-requestid': '2eb99fe0-520f-499c-9ff9-33c15323d9fb'},
  'RetryAttempts': 0},
 'Statement': '{"Sid":"allow_bedrock","Effect":"Allow","Principal":{"Service":"bedrock.amazonaws.com"},"Action":"lambda:InvokeFunction","Resource":"arn:aws:lambda:us-east-1:061051260563:function:restaurant-agent-mistral-lambda","Condition":{"ArnLike":{"AWS:SourceArn":"arn:aws:bedrock:us-east-1:061051260563:agent/PXINSLJAOL"}}}'}

### 4.7 Associate the Knowledge Base to the agent
Now we have created the Agent we can go ahead and associate the Knowledge Base we created earlier. 

In [29]:
response = bedrock_agent_client.associate_agent_knowledge_base(
    agentId=agent_id,
    agentVersion='DRAFT',
    description='Access the knowledge base when customers ask about the plates in the menu.',
    knowledgeBaseId=kb_id,
    knowledgeBaseState='ENABLED'
)

In [30]:
response

{'ResponseMetadata': {'RequestId': '25b47fa6-55ba-4e1d-acd7-942b8d8abbde',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 22 Apr 2025 15:28:49 GMT',
   'content-type': 'application/json',
   'content-length': '267',
   'connection': 'keep-alive',
   'x-amzn-requestid': '25b47fa6-55ba-4e1d-acd7-942b8d8abbde',
   'x-amz-apigw-id': 'JblTwGt1oAMEikA=',
   'x-amzn-trace-id': 'Root=1-6807b5b1-661fd56e2962106d74aa0077'},
  'RetryAttempts': 0},
 'agentKnowledgeBase': {'createdAt': datetime.datetime(2025, 4, 22, 15, 28, 49, 426412, tzinfo=tzutc()),
  'description': 'Access the knowledge base when customers ask about the plates in the menu.',
  'knowledgeBaseId': 'LD5AKXTEHU',
  'knowledgeBaseState': 'ENABLED',
  'updatedAt': datetime.datetime(2025, 4, 22, 15, 28, 49, 426412, tzinfo=tzutc())}}

### 4.8 Prepare the Agent and create an alias

Let's create a DRAFT version of the agent that can be used for internal testing.


In [31]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)
# Pause to make sure agent is prepared
time.sleep(30)

{'ResponseMetadata': {'RequestId': 'c3325435-338a-4fa7-b082-735d3c2a4315', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Tue, 22 Apr 2025 15:28:52 GMT', 'content-type': 'application/json', 'content-length': '119', 'connection': 'keep-alive', 'x-amzn-requestid': 'c3325435-338a-4fa7-b082-735d3c2a4315', 'x-amz-apigw-id': 'JblURHuHoAMESbg=', 'x-amzn-trace-id': 'Root=1-6807b5b4-519b04f35e29097273f0302e'}, 'RetryAttempts': 0}, 'agentId': 'PXINSLJAOL', 'agentStatus': 'PREPARING', 'agentVersion': 'DRAFT', 'preparedAt': datetime.datetime(2025, 4, 22, 15, 28, 52, 902380, tzinfo=tzutc())}


Now we can invoke the DRAFT version of your agent using the test alias id `TSTALIASID`

## 5. Test the Agent
Now that we've created the agent, let's use the `bedrock-agent-runtime` client to invoke this agent and perform some tasks. You can invoke your agent with the [`invoke_agent`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/invoke_agent.html) API

In [45]:
%%time
import json
from datetime import datetime

class DateTimeEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, datetime):
            return obj.isoformat()
        return super().default(obj)


def invokeAgent(query, session_id, enable_trace=False, session_state=dict()):
    end_session:bool = False
    
    # invoke the agent API
    agentResponse = bedrock_agent_runtime_client.invoke_agent(
        inputText=query,
        agentId=agent_id,
        agentAliasId="TSTALIASID", 
        sessionId=session_id,
        enableTrace=enable_trace, 
        endSession= end_session,
        sessionState=session_state
    )
    
    if enable_trace:
        logger.info(pprint.pprint(agentResponse))
    
    event_stream = agentResponse['completion']
    try:
        for event in event_stream:        
            if 'chunk' in event:
                data = event['chunk']['bytes']
                if enable_trace:
                    logger.info(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                end_event_received = True
                return agent_answer
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                if enable_trace:
                    logger.info(json.dumps(event['trace'], indent=2, cls=DateTimeEncoder))
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)

CPU times: user 31 μs, sys: 1 μs, total: 32 μs
Wall time: 32.9 μs


##### Invoke Agent to query Knowledge Base
Let's now use our support `invokeAgent` function to query our Knowledge Base with the Agent

In [46]:
%%time
import uuid
session_id:str = str(uuid.uuid1())
query = "What are the starters in the childrens menu?"
response = invokeAgent(query, session_id)
print(response)

Exception: ('unexpected event.', EventStreamError('An error occurred (dependencyFailedException) when calling the InvokeAgent operation: Dependency resource: received model timeout/error exception from Bedrock. Try the request again.'))

##### Invoke Agent to execute function from Action Group
Now let's test our Action Group functionality and create a new reservation

In [42]:
%%time
query = "Hi, I am Anna. I want to create a booking for 2 people, at 8pm on the 5th of May 2024."
response = invokeAgent(query, session_id)
print(response)

Exception: ('unexpected event.', EventStreamError('An error occurred (dependencyFailedException) when calling the InvokeAgent operation: Dependency resource: received model timeout/error exception from Bedrock. Try the request again.'))

##### Invoke Agent with prompt attribute

Great! We've used our agent to do the first reservation. However, often when booking tables in restaurants we are already logged in to systesm that know our names. How great would it be if our agent would know it as well?

To do so, we can use the session context to provide some attributes to our prompt. In this case we will provide it directly to the prompt using the [`promptSessionAttributes`](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-session-state.html) parameter. Let's also start a new session id so that our agent does not memorize our name.

In [43]:
%%time
session_id:str = str(uuid.uuid1())
query = "I want to create a booking for 2 people, at 8pm on the 5th of May 2024."
session_state = {
    "promptSessionAttributes": {
        "name": "John"
    }
}
response = invokeAgent(query, session_id, session_state=session_state)
print(response)

Exception: ('unexpected event.', EventStreamError('An error occurred (dependencyFailedException) when calling the InvokeAgent operation: Dependency resource: received model timeout/error exception from Bedrock. Try the request again.'))

##### Validating prompt attribute
Let's now use our session context to validate that the reservation was done under the correct name

In [47]:
%%time
query = "What was the name used in my last reservation?"
response = invokeAgent(query, session_id)
print(response)

Exception: ('unexpected event.', EventStreamError('An error occurred (dependencyFailedException) when calling the InvokeAgent operation: Dependency resource: received model timeout/error exception from Bedrock. Try the request again.'))

##### Retrieving information from the database in a new session

Next, let's confirm that our reservation system is working correctly.

In [ ]:
%%time
query = "I want to get the information for the previous booking"
response = invokeAgent(query, session_id)
print(response)

##### Canceling reservation

As plans change, we would now like to cancel the reservation we just did using our Agent for it.

In [ ]:
%%time
query = "Delete this booking"
response = invokeAgent(query, session_id)
print(response)

##### Handling context with PromptAttributes

With real-life applications, context is really important. We want to make reservations considering the current date and the days sorounding it. Amazon Bedrock Agents also allow you to provide temporal context for the agent with the prompt attributes. Let's test it with a reservation for tomorrow

In [ ]:
# retrieving today
from datetime import datetime
today = datetime.today().strftime('%b-%d-%Y')
today

In [ ]:
%%time
# reserving a table for tomorrow
session_id:str = str(uuid.uuid1())
query = "I want to create a booking for 2 people, at 8pm tomorrow."
session_state = {
    "promptSessionAttributes": {
        "name": "John",
        "today": today
    }
}
response = invokeAgent(query, session_id, session_state=session_state)
print(response)

##### Invoke Agent with Trace

Amazon Bedrock Agents also provides you with the details of steps being orchestrated by the Agent using the [Trace](https://docs.aws.amazon.com/bedrock/latest/userguide/trace-events.html). You can enable the trace during agent invocation. Let's now invoke the agent with the trace enabled

In [ ]:
%%time
session_id:str = str(uuid.uuid1())
query = "What are the desserts on the adult menu?"
response = invokeAgent(query, session_id, enable_trace=True)
print(response)

## 6. Clean-up 
Let's delete all the associated resources created to avoid unnecessary costs. 

In [ ]:
clean_up_resources(
    table_name, lambda_function, lambda_function_name, agent_action_group_response, agent_functions, 
    agent_id, kb_id, None
)

In [ ]:
# Delete the agent roles and policies
delete_agent_roles_and_policies(agent_name)

In [ ]:
# delete KB
knowledge_base.delete_kb(delete_s3_bucket=True, delete_iam_roles_and_policies=True)

## Next steps

Congratulations! You've now created a restaurant assistant using Mistral Large on Bedrock Agents. As next steps, try creating your own application and further updating your agent's prompts